In [2]:
import numpy as np
import pandas as pd

In [146]:
class Map(object): 
    def __init__(self,size,learning_rate):
        self.size = size
        self.learning_rate = learning_rate
        self.neighborhood_radius = np.sqrt(1/2.0)*size #diagonal of the map
        self.neurons = np.random.randn(size,size,2)
    
    def find_bmu(self, input_node): #find best matching unit
        min_distance = np.inf
        bum_index = ()
        for row in xrange(self.size):
            for col in xrange(self.size):
                distance = Euclidean_distance(self.neurons[row,col], input_node)
                #print distance
                if distance < min_distance: 
                    min_distance = distance
                    bum_index =(row,col)
                    #print bum_index
        return bum_index
    
    def weight_update(self): 
        return 
    
    def learning_rate_update(self):
        return 
    
    def neighborhood_function(self):
        return
    
    def radius_update(self):
        return 
    
    def Kohonen_learning(self, training_set, iteration=1000): 
        
        return 
        
        

In [147]:
def Euclidean_distance(pointA,pointB):
    #print pointA, pointB
    return np.linalg.norm(pointA-pointB)

In [148]:
m = Map(3,10)

In [149]:
m.neurons

array([[[ 0.66233776, -1.55752666],
        [ 0.12146938, -0.59912606],
        [-0.73402657, -0.05014694]],

       [[-0.80024054,  0.93123677],
        [ 0.45486283, -1.90427234],
        [-0.49247375,  0.45648215]],

       [[ 1.4810063 , -0.95449116],
        [ 0.81608337, -0.07783733],
        [ 0.63001633,  0.37771138]]])

In [150]:
input_node = np.array([1.8, -1])
input_node

array([ 1.8, -1. ])

In [152]:
i = m.find_bmu(input_node)

In [153]:
m.neurons[i]

array([ 1.4810063 , -0.95449116])

In [47]:
np.random.randn(1,2,3)

array([[[ 0.4913067 ,  1.5393302 , -0.67129326],
        [-0.13727846,  0.46435813,  0.13712856]]])

In [51]:
np.random.randn(2,1,3)

array([[[-1.5167157 , -1.70312587,  1.9992699 ]],

       [[-0.57756045, -0.20065439,  1.64940515]]])

In [4]:
np.sqrt(1/2.0)

0.70710678118654757

In [11]:
t = np.random.randn(3,3)
t

array([[ 0.22972892, -0.65749864,  0.05444808],
       [ 0.39569672,  0.09140442, -1.47999884],
       [ 0.64572364,  1.14131451, -0.41028335]])

In [14]:
t[1,2] = np.random.

In [15]:
t

array([[  2.29728924e-01,  -6.57498642e-01,   5.44480757e-02],
       [  3.95696723e-01,   9.14044203e-02,   9.99000000e+02],
       [  6.45723639e-01,   1.14131451e+00,  -4.10283355e-01]])

In [19]:
t.apply(lambda x: np.random.randn(1,2))

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [65]:
t = np.random.randn(4,4,2)

In [68]:
t

array([[[-0.03989358, -0.02505185],
        [ 1.49336349,  0.60036155],
        [ 2.09108263,  1.15908961],
        [ 1.09469601, -0.06512583]],

       [[-0.59340555,  0.8200864 ],
        [-1.12821295,  1.20069078],
        [ 0.60873641,  1.83164727],
        [-0.96750927,  1.22814561]],

       [[-0.37776588, -0.29306753],
        [ 0.73834805,  1.03463801],
        [ 1.5152398 ,  0.56557662],
        [ 0.18499532, -0.30902274]],

       [[-0.8909503 ,  0.84428083],
        [ 0.06893675,  1.38124708],
        [-0.08330903,  0.20468468],
        [-0.61800443,  0.07574253]]])

In [100]:
t[1,2]

array([ 0.60873641,  1.83164727])

In [102]:
x = [1,2]
t[x]

array([[[-0.59340555,  0.8200864 ],
        [-1.12821295,  1.20069078],
        [ 0.60873641,  1.83164727],
        [-0.96750927,  1.22814561]],

       [[-0.37776588, -0.29306753],
        [ 0.73834805,  1.03463801],
        [ 1.5152398 ,  0.56557662],
        [ 0.18499532, -0.30902274]]])

In [91]:
x = t[0,1].reshape(2,1)
y = t[1,2].reshape(2,1)

In [93]:
y

array([[ 0.60873641],
       [ 1.83164727]])

In [90]:
x

array([[ 1.49336349],
       [ 0.60036155]])

1.5161232127655642

In [20]:
f = lambda x : np.random.randn(1,2)

In [21]:
f2 = np.vectorize(f)

In [41]:
t

array([[  2.29728924e-01,  -6.57498642e-01,   5.44480757e-02],
       [  3.95696723e-01,   9.14044203e-02,   9.99000000e+02],
       [  6.45723639e-01,   1.14131451e+00,  -4.10283355e-01]])

In [43]:
for x in t:
    for y in x:
        y = 999 